In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import math
import warnings

# Train

In [2]:
from config import get_config, get_weights_file_path
from train import train_model, get_ds, get_model, run_validation, majority_vote

In [3]:
# if __name__ == "__main__":
#   warnings.filterwarnings("ignore")
#   config = get_config()
  # train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config=config["model_1"])
#   for _, value in config.items():
#     train_model(value, train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt)
if __name__ == "__main__":
  warnings.filterwarnings("ignore")
  config = get_config()
  train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config=config["model_1"])
  for _, value in config.items():
    train_model(value, train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt)

path tokenizer :  /home/zaens/Codes/Skripsi/Codes/Machine Translation Majority vote ensemble/tokenizer_indonesian.json
tokenizer file ada
path tokenizer :  /home/zaens/Codes/Skripsi/Codes/Machine Translation Majority vote ensemble/tokenizer_english.json
tokenizer file ada
path tokenizer :  /home/zaens/Codes/Skripsi/Codes/Machine Translation Majority vote ensemble/tokenizer_indonesian.json
tokenizer file ada
path tokenizer :  /home/zaens/Codes/Skripsi/Codes/Machine Translation Majority vote ensemble/tokenizer_english.json
tokenizer file ada
max length of source sentence 34
max length of target sentence 34
using device :  cuda
Preloading model weights/model_1/tmodel_99.pt


Processing  epoch 100: 100%|██████████| 245/245 [00:19<00:00, 12.54it/s, Loss=1.691]


--------------------------------------------------------------------------------
Source: di bawah rencana yang diumumkan pada hari rabu di washington gm akan menyediakan pabrik dow dengan truk yang berisi peralatan konversi sel bahan bakar
Target: under the plan being announced wednesday in washington gm will provide dow manufacturing plants with trucks containing fuel cell conversion equipment
Predicted: under the plan being announced wednesday in washington gm will provide dow manufacturing plants with trucks containing fuel cell conversion equipment
--------------------------------------------------------------------------------
Source: puretunes juga telah kehilangan kesepakatan distribusi dengan perusahaan perangkat lunak swapping grokster populer
Target: puretunes has also lost a distribution deal with the popular grokster fileswapping software company
Predicted: puretunes has also lost a distribution deal with the popular grokster software company


Processing  epoch 101: 100%|██████████| 245/245 [00:18<00:00, 13.21it/s, Loss=1.814]


--------------------------------------------------------------------------------
Source: di bawah rencana yang diumumkan pada hari rabu di washington gm akan menyediakan pabrik dow dengan truk yang berisi peralatan konversi sel bahan bakar
Target: under the plan being announced wednesday in washington gm will provide dow manufacturing plants with trucks containing fuel cell conversion equipment
Predicted: under the plan being announced wednesday in washington gm will provide dow manufacturing plants with trucks containing fuel cell conversion equipment
--------------------------------------------------------------------------------
Source: puretunes juga telah kehilangan kesepakatan distribusi dengan perusahaan perangkat lunak swapping grokster populer
Target: puretunes has also lost a distribution deal with the popular grokster fileswapping software company
Predicted: puretunes has also lost a distribution deal with the popular grokster software company


...

--------------------------------------------------------------------------------
Source: di bawah rencana yang diumumkan pada hari rabu di washington gm akan menyediakan pabrik dow dengan truk yang berisi peralatan konversi sel bahan bakar
Target: under the plan being announced wednesday in washington gm will provide dow manufacturing plants with trucks containing fuel cell conversion equipment
Predicted: under the plan being announced wednesday in washington gm will provide dow manufacturing plants with trucks containing fuel cell conversion equipment
--------------------------------------------------------------------------------
Source: puretunes juga telah kehilangan kesepakatan distribusi dengan perusahaan perangkat lunak swapping grokster populer
Target: puretunes has also lost a distribution deal with the popular grokster fileswapping software company
Predicted: puretunes has also lost a distribution deal with the popular grokster software company


Processing  epoch 120: 100%|██████████| 245/245 [00:19<00:00, 12.83it/s, Loss=1.635]


--------------------------------------------------------------------------------
Source: di bawah rencana yang diumumkan pada hari rabu di washington gm akan menyediakan pabrik dow dengan truk yang berisi peralatan konversi sel bahan bakar
Target: under the plan being announced wednesday in washington gm will provide dow manufacturing plants with trucks containing fuel cell conversion equipment
Predicted: under the plan announced wednesday in washington gm will provide dow manufacturing plants with trucks containing fuel cell conversion equipment
--------------------------------------------------------------------------------
Source: puretunes juga telah kehilangan kesepakatan distribusi dengan perusahaan perangkat lunak swapping grokster populer
Target: puretunes has also lost a distribution deal with the popular grokster fileswapping software company
Predicted: puretunes has also lost a distribution deal with the popular grokster software company


Processing  epoch 121:  87%|████████▋ | 213/245 [00:16<00:02, 12.71it/s, Loss=1.591]


KeyboardInterrupt: 

# Inference

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using device : ", device)
config = get_config()
train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config=config["model_1"])
models = {}
for model_name, value in config.items():
  print("===================================")
  print("Model : ", model_name)
  model = get_model(config=value, vocab_src_len=tokenizer_src.get_vocab_size(), vocab_tgt_len=tokenizer_tgt.get_vocab_size()).to(device=device)
  model_filename = get_weights_file_path(value, value["preload"])
  state = torch.load(model_filename)
  model.load_state_dict(state["model_state_dict"])
  models[model_name] = model
  run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, value["seq_len"], device, lambda msg: print(msg), 0, None, num_examples=3)

# model = get_model(config=config, vocab_src_len=tokenizer_src.get_vocab_size(), vocab_tgt_len=tokenizer_tgt.get_vocab_size()).to(device=device)

# model_filename = get_weights_file_path(config, f"99")
# state = torch.load(model_filename)
# model.load_state_dict(state["model_state_dict"])

using device :  cuda
path tokenizer :  /home/zaens/Codes/Skripsi/Codes/Machine Translation Majority vote ensemble/tokenizer_indonesian.json
tokenizer file ada
path tokenizer :  /home/zaens/Codes/Skripsi/Codes/Machine Translation Majority vote ensemble/tokenizer_english.json
tokenizer file ada
path tokenizer :  /home/zaens/Codes/Skripsi/Codes/Machine Translation Majority vote ensemble/tokenizer_indonesian.json
tokenizer file ada
path tokenizer :  /home/zaens/Codes/Skripsi/Codes/Machine Translation Majority vote ensemble/tokenizer_english.json
tokenizer file ada
max length of source sentence 34
max length of target sentence 34
Model :  model_1
--------------------------------------------------------------------------------
Source: pria itu tidak di atas es tetapi terjebak di jeram bergoyang eddy sekitar 250 kaki dari pantai
Target: the man wasnt on the ice but trapped in the rapids swaying in an eddy about 250 feet from the shore
Predicted: the man wasnt on the ice but trapped in the in 

In [5]:
test = [
    ("saya suka kamu", "i like you"),
    ("saya tidak suka kamu", "i don't like you"),
    ("saya sangat suka kamu", "i really like you"),
    ("saya sangat tidak suka kamu", "i really don't like you"),
    ("saya sangat sangat suka kamu", "i really really like you"),
    ("saya sangat sangat tidak suka kamu", "i really really don't like you"),
    ("saya sangat sangat sangat suka kamu", "i really really really like you"),
    ("saya sangat sangat sangat tidak suka kamu", "i really really really don't like you"),
]

train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config=config["model_1"])
# src_ids = tokenizer_src.encode("saya sedang memancing di taman").ids
# print("src_ids : ", src_ids)
results = {
  "src_text" : [],
  "expected" : [],
  "model_1" : [],
  "model_2" : [],
  "model_3" : [],
  "majority_vote" : [],
}

for model_name, model in models.items():
  print("===================================")
  print("Model : ", model_name)
  src_text, expected, predicted = majority_vote(model, val_dataloader, tokenizer_src, tokenizer_tgt, config["model_1"]["seq_len"], device, lambda msg: print(msg), 0, None, num_examples=20)
  if model_name == "model_1":
    results["expected"].append(expected)
    results["src_text"].append(src_text)
  results[model_name].append(predicted)

path tokenizer :  /home/zaens/Codes/Skripsi/Codes/Machine Translation Majority vote ensemble/tokenizer_indonesian.json
tokenizer file ada
path tokenizer :  /home/zaens/Codes/Skripsi/Codes/Machine Translation Majority vote ensemble/tokenizer_english.json
tokenizer file ada
path tokenizer :  /home/zaens/Codes/Skripsi/Codes/Machine Translation Majority vote ensemble/tokenizer_indonesian.json
tokenizer file ada
path tokenizer :  /home/zaens/Codes/Skripsi/Codes/Machine Translation Majority vote ensemble/tokenizer_english.json
tokenizer file ada
max length of source sentence 34
max length of target sentence 34
Model :  model_1
--------------------------------------------------------------------------------
Source: saham ahold naik 12 sen atau 23 ​​persen menjadi 535 euro di amsterdam
Target: shares of ahold rose 12 cents or 23 percent to 535 euros in amsterdam
Predicted: shares of ahold rose 12 cents or 23 percent to euros in amsterdam
--------------------------------------------------------

In [6]:
print(results["src_text"][:5])
print(results["expected"][:5])
print(results["model_1"][:5])
print(results["model_2"][:5])
print(results["model_3"][:5])

[['saham ahold naik 12 sen atau 23 \u200b\u200bpersen menjadi 535 euro di amsterdam', 'bahkan ketika pertukaran mendeteksi pelanggaran seperti itu itu sering gagal mengambil tindakan disiplin yang tepat terhadap individu atau perusahaan', 'tarif hipotek di seluruh negeri turun lagi minggu ini tingkat kesembilan tahun ini mencapai titik terendah sepanjang masa', 'indeks standard poors 500 naik 175 poin atau 018 persen menjadi 97768', 'sersan ernest bucklew 33 pulang dari irak untuk mengubur ibunya di pennsylvania', 'prajurit amerika pertama kali dipilih sebagai times person of the year selama perang korea pada tahun 1950', 'duque tiba dengan foale dan kaleri tetapi akan kembali ke bumi dengan lu dan malenchenko', 'benchmark 10tahun catatan us10yt rr naik 1132 untuk hasil 325 persen', 'dia berencana untuk makan malam dengan pasukan di markas militer as kosovo camp bondsteel', 'dia telah tampil di luar kuba pada beberapa kesempatan termasuk pertunjukan di amerika serikat tahun lalu', 'car

In [7]:
print(len(results["src_text"][0]))
print(len(results["expected"][0]))
print(len(results["model_1"][0]))
print(len(results["model_2"][0]))
print(len(results["model_3"][0]))


20
20
20
20
20


In [8]:
for text_idx in range(len(results["src_text"][0])):
  print("===================================")
  # print("Source : ", results["src_text"][i])
  # print("Expected : ", results["expected"][i])
  
  # for model_name in ["model_1", "model_2", "model_3"]:
  #   print(f"{model_name} : ", results[model_name][i])
  model_1 = results["model_1"][0][text_idx]
  model_2 = results["model_2"][0][text_idx]
  model_3 = results["model_3"][0][text_idx]
  expected = results["expected"][0][text_idx]
  print("model_1 : ")
  print(model_1)
  print("model_2 : ")
  print(model_2)
  print("model_3 : ")
  print(model_3)
  print("expected : ")
  print(expected)
  model_1 = model_1.split()
  model_2 = model_2.split()
  model_3 = model_3.split()
  expected = expected.split()
  majority_words = []
  max_len = max(len(model_1), len(model_2), len(model_3), len(expected))
  for i in range(max_len):
    words = []
    if i < len(model_1):
      words.append(model_1[i])
    if i < len(model_2):
      words.append(model_2[i])
    if i < len(model_3):
      words.append(model_3[i])
    # print("word : ", words)
    if words == []:
      continue
    majority_word = max(set(words), key = words.count)
    # print("majority_word : ", majority_word, "expected : ", expected[i] if i < len(expected) else "")
    majority_words.append(majority_word)
  majority_words = " ".join(majority_words)
  print("Majority vote : ")
  print(majority_words)
    # results["majority_vote"].append(majority_word)



model_1 : 
shares of ahold rose 12 cents or 23 percent to euros in amsterdam
model_2 : 
shares of ahold rose 12 cents or 23 percent to euros in amsterdam
model_3 : 
shares of ahold rose 12 cents or 23 percent to euros in amsterdam
expected : 
shares of ahold rose 12 cents or 23 percent to 535 euros in amsterdam
Majority vote : 
shares of ahold rose 12 cents or 23 percent to euros in amsterdam
model_1 : 
even when the exchange detected such violations it often failed to take appropriate disciplinary actions against individuals or firms
model_2 : 
even when the exchange detected such violations it often failed to take appropriate disciplinary actions against individuals or firms
model_3 : 
even when the exchange detected such violations it often failed to take appropriate disciplinary actions against individuals or firms
expected : 
even when the exchange detected such violations it often failed to take appropriate disciplinary actions against individuals or firms
Majority vote : 
even w

In [11]:
run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config["model_1"]["seq_len"], device, lambda msg: print(msg), 0, None, num_examples=10)

--------------------------------------------------------------------------------
Source: saham ahold naik 12 sen atau 23 ​​persen menjadi 535 euro di amsterdam
Target: shares of ahold rose 12 cents or 23 percent to 535 euros in amsterdam
Predicted: shares of ahold rose 12 cents or 23 percent to euros in amsterdam
--------------------------------------------------------------------------------
Source: bahkan ketika pertukaran mendeteksi pelanggaran seperti itu itu sering gagal mengambil tindakan disiplin yang tepat terhadap individu atau perusahaan
Target: even when the exchange detected such violations it often failed to take appropriate disciplinary actions against individuals or firms
Predicted: even when the exchange detected such violations it often failed to take appropriate disciplinary actions against individuals or firms
--------------------------------------------------------------------------------
Source: tarif hipotek di seluruh negeri turun lagi minggu ini tingkat kesembil